In [2]:
import torch
import torch.nn as nn
import torch.fft
import numpy as np

In [21]:
import logging

# Настройка логгера
def setup_logger(log_file_path):
    logger = logging.getLogger("DFDGCN_Logger")
    logger.setLevel(logging.INFO)
    
    # Формат сообщения
    formatter = logging.Formatter('%(asctime)s - %(message)s')
    
    # Файловый хэндлер для записи в txt файл
    file_handler = logging.FileHandler(log_file_path)
    file_handler.setFormatter(formatter)
    logger.addHandler(file_handler)
    
    # # Консольный хэндлер для вывода в консоль (опционально)
    # console_handler = logging.StreamHandler()
    # console_handler.setFormatter(formatter)
    # logger.addHandler(console_handler)
    
    return logger

# Функция для логирования shape тензоров
def log_tensor_shape(logger, tensor, tensor_name):
    if tensor is not None:
        logger.info(f"{tensor_name} shape: {tensor.shape}")
    else:
        logger.info(f"{tensor_name} is None")

In [22]:
class DFDGCN(nn.Module):
    def __init__(self, num_nodes, embedding_dim, num_weeks, num_days, K, seq_len, logger):
        super(DFDGCN, self).__init__()
        self.num_nodes = num_nodes
        self.embedding_dim = embedding_dim
        self.K = K
        self.seq_len = seq_len
        self.logger = logger  # Добавляем логгер в модель

        # Identity Embedding
        self.identity_embedding = nn.Embedding(num_nodes, embedding_dim)
        
        # Time Embeddings
        self.week_embedding = nn.Embedding(num_weeks, embedding_dim)
        self.day_embedding = nn.Embedding(num_days, embedding_dim)
        
        # Linear layers for Fourier transformed data and time embeddings
        self.W_F = nn.Linear(1, embedding_dim)  # Adjusted to match input dimension
        self.W_T = nn.Linear(2*embedding_dim, embedding_dim)
        
        # 1x1 Convolution
        self.conv1x1 = nn.Conv1d(3*embedding_dim, embedding_dim, kernel_size=1)
        
        # Adjacency matrix learning
        self.W_adj = nn.Parameter(torch.randn(embedding_dim, embedding_dim))
        
        # Graph convolution weights
        self.W_k1 = [nn.Linear(seq_len, embedding_dim) for _ in range(K+1)]
        self.W_k2 = [nn.Linear(seq_len, embedding_dim) for _ in range(K+1)]
        self.W_k3 = [nn.Linear(seq_len, embedding_dim) for _ in range(K+1)]
        
        # Output layer to match target dimension
        self.output_layer = nn.Linear(embedding_dim, seq_len)

    def apply_fft(self, x):
        # x shape: [batch_size, num_nodes, seq_len]
        fft_x = torch.fft.fft(x, dim=-1)
        fft_x = fft_x.abs()  # Magnitude of FFT
        log_tensor_shape(self.logger, fft_x, "FFT(X_t)")
        return fft_x

    def compute_DE_t(self, x_t, week_idx, day_idx):
        # x_t shape: [batch_size, num_nodes, seq_len]
        fft_x_t = self.apply_fft(x_t)  # [batch_size, num_nodes, seq_len]
        fft_x_t = fft_x_t.mean(dim=-1)  # [batch_size, num_nodes]
        log_tensor_shape(self.logger, fft_x_t, "FFT(X_t).mean")
        
        # Identity Embedding
        E_t = self.identity_embedding(torch.arange(self.num_nodes, device=x_t.device))  # [num_nodes, embedding_dim]
        log_tensor_shape(self.logger, E_t, "Identity Embedding")
        
        # Time Embeddings
        T_W = self.week_embedding(week_idx)  # [batch_size, embedding_dim]
        T_D = self.day_embedding(day_idx)    # [batch_size, embedding_dim]
        T = torch.cat((T_W, T_D), dim=-1)    # [batch_size, 2*embedding_dim]
        log_tensor_shape(self.logger, T, "Time Embedding (concat)")
        T = self.W_T(T)                      # [batch_size, embedding_dim]
        log_tensor_shape(self.logger, T, "Time Embedding (W_T)")
        
        # Fourier Embedding
        F_t = fft_x_t.unsqueeze(-1)          # [batch_size, num_nodes, 1]
        log_tensor_shape(self.logger, F_t, "FFT(X_t).unsqueeze")
        F_t = self.W_F(F_t).squeeze(-1)      # [batch_size, num_nodes, embedding_dim]
        log_tensor_shape(self.logger, F_t, "Fourier Embedding (W_F)")
        
        # Expand E_t and T to match dimensions
        E_t_expanded = E_t.unsqueeze(0).expand(x_t.size(0), self.num_nodes, self.embedding_dim)  # [batch_size, num_nodes, embedding_dim]
        log_tensor_shape(self.logger, E_t_expanded, "Identity Embedding (expanded)")
        T_expanded = T.unsqueeze(1).expand(x_t.size(0), self.num_nodes, self.embedding_dim)      # [batch_size, num_nodes, embedding_dim]
        log_tensor_shape(self.logger, T_expanded, "Time Embedding (expanded)")
        
        # Concatenate
        DE_t = torch.cat((F_t, E_t_expanded, T_expanded), dim=-1)  # [batch_size, num_nodes, 3*embedding_dim]
        log_tensor_shape(self.logger, DE_t, "DE_t (concat)")
        DE_t = DE_t.permute(0, 2, 1)  # [batch_size, 3*embedding_dim, num_nodes]
        log_tensor_shape(self.logger, DE_t, "DE_t (permute)")
        DE_t = self.conv1x1(DE_t)  # [batch_size, embedding_dim, num_nodes]
        log_tensor_shape(self.logger, DE_t, "DE_t (conv1x1)")
        DE_t = DE_t.permute(0, 2, 1)  # [batch_size, num_nodes, embedding_dim]
        log_tensor_shape(self.logger, DE_t, "DE_t (final)")
        return DE_t

    def compute_A_D(self, DE_t):
        A = torch.relu(DE_t @ self.W_adj @ DE_t.transpose(1, 2))  # [batch_size, num_nodes, num_nodes]
        log_tensor_shape(self.logger, A, "A_D (relu)")
        A = torch.softmax(A, dim=-1)  # [batch_size, num_nodes, num_nodes]
        log_tensor_shape(self.logger, A, "A_D (softmax)")
        return A

    def graph_convolution(self, X_t, P, A_adt, A_D):
        Z_t = 0
        for k in range(self.K+1):
            P_k = torch.matrix_power(P, k)  # [num_nodes, num_nodes]
            log_tensor_shape(self.logger, P_k, f"P^{k}")
            A_adt_k = torch.matrix_power(A_adt, k)  # [num_nodes, num_nodes]
            log_tensor_shape(self.logger, A_adt_k, f"A_adt^{k}")
            A_D_k = torch.matrix_power(A_D, k)  # [batch_size, num_nodes, num_nodes]
            log_tensor_shape(self.logger, A_D_k, f"A_D^{k}")
            
            term1 = (P_k @ X_t) @ self.W_k1[k].weight.t()  # [batch_size, num_nodes, embedding_dim]
            log_tensor_shape(self.logger, term1, f"Term1 (P_k @ X_t)")
            term2 = (A_adt_k @ X_t) @ self.W_k2[k].weight.t()  # [batch_size, num_nodes, embedding_dim]
            log_tensor_shape(self.logger, term2, f"Term2 (A_adt_k @ X_t)")
            term3 = (A_D_k @ X_t) @ self.W_k3[k].weight.t()  # [batch_size, num_nodes, embedding_dim]
            log_tensor_shape(self.logger, term3, f"Term3 (A_D_k @ X_t)")
            Z_t += term1 + term2 + term3
        log_tensor_shape(self.logger, Z_t, "Z_t (graph convolution)")
        return Z_t

    def forward(self, X_t, week_idx, day_idx, P, A_adt):
        log_tensor_shape(self.logger, X_t, "X_t (input)")
        DE_t = self.compute_DE_t(X_t, week_idx, day_idx)
        A_D = self.compute_A_D(DE_t)
        Z_t = self.graph_convolution(X_t, P, A_adt, A_D)
        Z_t = self.output_layer(Z_t)  # [batch_size, num_nodes, seq_len]
        log_tensor_shape(self.logger, Z_t, "Z_t (output)")
        return Z_t

In [24]:
# Настройка логгера
logger = setup_logger("dfdgnn_log.txt")

# Параметры
num_nodes = 100
seq_len = 12
pre_len = 3
batch_size = 32
embedding_dim = 10
num_weeks = 7
num_days = 24
K = 2

# Генерация данных
X_t = torch.randn(batch_size, num_nodes, seq_len)
y = torch.randn(batch_size, num_nodes, pre_len)
week_idx = torch.randint(0, num_weeks, (batch_size,))
day_idx = torch.randint(0, num_days, (batch_size,))
P = torch.rand(num_nodes, num_nodes)
A_adt = torch.rand(num_nodes, num_nodes)

# Создание модели
model = DFDGCN(num_nodes, embedding_dim, num_weeks, num_days, K, seq_len, logger)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Обучение
num_epochs = 5
for epoch in range(num_epochs):
    optimizer.zero_grad()
    output = model(X_t, week_idx, day_idx, P, A_adt)
    loss = criterion(output, y)
    loss.backward()
    optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

2024-12-14 13:50:09,208 - X_t (input) shape: torch.Size([32, 100, 12])
2024-12-14 13:50:09,208 - X_t (input) shape: torch.Size([32, 100, 12])
2024-12-14 13:50:09,223 - FFT(X_t) shape: torch.Size([32, 100, 12])
2024-12-14 13:50:09,223 - FFT(X_t) shape: torch.Size([32, 100, 12])
2024-12-14 13:50:09,229 - FFT(X_t).mean shape: torch.Size([32, 100])
2024-12-14 13:50:09,229 - FFT(X_t).mean shape: torch.Size([32, 100])
2024-12-14 13:50:09,232 - Identity Embedding shape: torch.Size([100, 10])
2024-12-14 13:50:09,232 - Identity Embedding shape: torch.Size([100, 10])
2024-12-14 13:50:09,235 - Time Embedding (concat) shape: torch.Size([32, 20])
2024-12-14 13:50:09,235 - Time Embedding (concat) shape: torch.Size([32, 20])
2024-12-14 13:50:09,237 - Time Embedding (W_T) shape: torch.Size([32, 10])
2024-12-14 13:50:09,237 - Time Embedding (W_T) shape: torch.Size([32, 10])
2024-12-14 13:50:09,241 - FFT(X_t).unsqueeze shape: torch.Size([32, 100, 1])
2024-12-14 13:50:09,241 - FFT(X_t).unsqueeze shape: t

RuntimeError: The size of tensor a (12) must match the size of tensor b (3) at non-singleton dimension 2